In [133]:
#IMPORT MODULES
import pandas as pd
import numpy as np
import time
# SENTIMENT ANALYSIS USING VADER
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, precision_score, recall_score, classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [134]:
#READ THE DATA FILE
df1 = pd.read_csv("../Datasets/survey_results_public.csv")

In [135]:
#ADD COLUMNS HERE

#Employment Column Additions
column_name = "Employment"
st = set()
for i in range (0,len(df1)):
    value = str(df1[column_name].iloc[i])
    if(value == "nan"):continue
    l = value.split(";")
    for ele in l:
        st.add(ele)
for ele in st:
    df1[ele] = 0
    
for i in range (0,len(df1)):
    value = str(df1[column_name].iloc[i])
    if(value == "nan"):continue
    l = value.split(";")
    for ele in l:
        df1.loc[i,ele] = 1

#Countries filtered out if not India
df1 = df1[df1["Industry"] == "Information Services, IT, Software Development, or other Technology"]
#Industries filtered out if not Information Services, IT, Software Development, or other Technology
df1 = df1[df1["Country"] == "India"]

#Number of Languages known
column_name = "LanguageHaveWorkedWith"
def languagecount(row):
    value = str(row[column_name]).split(";")
    if(value[0] == "nan"):return 0
    return len(value)
df1["NumberOfLanguagesKnown"] = df1.apply(languagecount,axis = 1)

# # #Number of Databases known
column_name = "DatabaseHaveWorkedWith"
def languagecount(row):
    value = str(row[column_name]).split(";")
    if(value[0] == "nan"):return 0
    return len(value)
df1["NumberOfDatabasesKnown"] = df1.apply(languagecount,axis = 1)

In [136]:
columns_selected =  [
   'AISelect',
   'OrgSize',
   'DevType',
   'YearsCode',
   'WorkExp', 
   'YearsCodePro',
   "RemoteWork",
   'Currency',
   "EdLevel",
   "ConvertedCompYearly",
   "NumberOfDatabasesKnown",
   "NumberOfLanguagesKnown"
]

train_columns = [
   'AISelect',
   'OrgSize',
   'DevType',
   "RemoteWork",
   'Currency',
   "EdLevel",
   "ExperienceCategory",
   "YearsCodeCategory",
   "YearsCodeProCategory",
   "NumberOfDatabasesKnown",
   "NumberOfLanguagesKnown"
]

for ele in st:
    train_columns.append(ele)
    columns_selected.append(ele)

df1 = df1[columns_selected]

df1 = df1.dropna()

In [137]:
#CATEGORISE COLUMNS  INTO MAJORITY VALUES AND 'OTHER'
def shorten_categories(categories, cutoff):
    categorical_map = {}
    for i in range(len(categories)):
        if categories.values[i] >= cutoff:
            categorical_map[categories.index[i]] = categories.index[i]
        else:
            categorical_map[categories.index[i]] = 'Other'
    return categorical_map


currency_map = shorten_categories(df1.Currency.value_counts(), 400)
df1['Currency'] = df1['Currency'].map(currency_map)

In [138]:
#CATEGORISE THE WORK EXPERIENCE INTO BINS
bins = [0, 2, 5, 10, 20, 30, 40, 50, float('inf')]  # Define custom bin edges
labels = [0, 1, 2, 3, 4, 5, 6, 7]  # Define labels

# Create a new column with the categories
df1['ExperienceCategory'] = pd.cut(df1['WorkExp'], bins=bins, labels=labels)

In [139]:
#CATEGORISE LESS THAN 1 YEAR AS 0 AND MORE THAN 50 AS 51 FOR YEARS OF CODE
df1['YearsCode'] = df1['YearsCode'].replace("Less than 1 year", 0)
df1['YearsCode'] = df1['YearsCode'].replace("More than 50 years", 51)

df1['YearsCodePro'] = df1['YearsCodePro'].replace("Less than 1 year", 0)
df1['YearsCodePro'] = df1['YearsCodePro'].replace("More than 50 years", 51)

In [140]:
#CATEGORISE YEARS OF CODE INTO BINS
bins = [0, 2, 5, 10, 20, 30, 40, 50, float('inf')]  # Define custom bin edges
labels = [0, 1, 2, 3, 4, 5, 6, 7]  # Define labels

# Create a new column with the categories
df1["YearsCode"] = df1["YearsCode"].astype(int)
df1["YearsCodePro"] = df1["YearsCodePro"].astype(int)
df1['YearsCodeCategory'] = pd.cut(df1['YearsCode'], bins=bins, labels=labels)
df1['YearsCodeProCategory'] = pd.cut(df1['YearsCodePro'], bins=bins, labels=labels)

In [141]:
#FILTER THE DATAFRAME
df1 = df1[df1["ConvertedCompYearly"] <= 250000]
df1 = df1[df1["ConvertedCompYearly"] >= 100]

In [142]:
len(df1)

878

In [143]:
#LABEL ENCODE THE COLUMNS

df_LE = df1
df_LE = df_LE.dropna()

for i in train_columns:
    if i == "ConvertedCompYearly":
        continue
    le = LabelEncoder()
    df_LE[i] = le.fit_transform(df_LE[i])

X = df_LE[train_columns]

Y = df_LE["ConvertedCompYearly"]

C:\Users\Siddharth Shah\AppData\Local\Temp\ipykernel_16984\2931493265.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_LE[i] = le.fit_transform(df_LE[i])
C:\Users\Siddharth Shah\AppData\Local\Temp\ipykernel_16984\2931493265.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_LE[i] = le.fit_transform(df_LE[i])
C:\Users\Siddharth Shah\AppData\Local\Temp\ipykernel_16984\2931493265.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [144]:
#TRAIN-TEST SPLIT
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.1, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [145]:
#INITIALISE MODELS
classifiers = {}

# Linear Regression
classifiers["LinearRegression"] = LinearRegression()

# Decision Tree
classifiers["DecisionTree"] = DecisionTreeRegressor()

# Random Forest
classifiers["RandomForest"] = RandomForestRegressor()

# XGBoost
classifiers["XGBoost"] = XGBRegressor()

# # LightGBM
# classifiers["LightGBM"] = LGBMRegressor()

# # CatBoost
# classifiers["CatBoost"] = CatBoostRegressor(silent=True)

In [146]:
#CHECK IMPORTANCE
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(X, Y)
feature_importances = model.feature_importances_

map_ = {}

for i, a in enumerate(X):
    map_[a] = feature_importances[i]
    
sorted(map_.items(), key = lambda x: x[1])

[('Retired', 0.0),
 ('Student, full-time', 0.0),
 ('Not employed, and not looking for work', 0.0),
 ('Not employed, but looking for work', 0.0),
 ('I prefer not to say', 0.0),
 ('Student, part-time', 0.0),
 ('Employed, part-time', 0.00214794577717307),
 ('Independent contractor, freelancer, or self-employed',
  0.012831491571007029),
 ('Employed, full-time', 0.02011554988622515),
 ('RemoteWork', 0.03868893671981635),
 ('AISelect', 0.03925471250129547),
 ('Currency', 0.04004045577522342),
 ('EdLevel', 0.04582292225088376),
 ('YearsCodeProCategory', 0.059399802484508823),
 ('OrgSize', 0.08675018923817798),
 ('NumberOfDatabasesKnown', 0.08862706948739536),
 ('YearsCodeCategory', 0.11524388155822743),
 ('DevType', 0.12505181691654751),
 ('ExperienceCategory', 0.15008622855643852),
 ('NumberOfLanguagesKnown', 0.17593899727708037)]

In [147]:
# Create an empty DataFrame to store the results
columns = ['Model', 'Run Time (minutes)', 'MAE', 'MSE', 'RMSE', 'R2']
df_models = pd.DataFrame(columns=columns)

# Loop through your regression models
for key, clf in classifiers.items():
    # STARTING TIME
    start_time = time.time()
    # TRAIN CLASSIFIER ON TRAINING DATA
    clf.fit(X_train_scaled, y_train)
    # MAKE PREDICTIONS USING CURRENT CLASSIFIER
    predictions = clf.predict(X_test_scaled)
    # CALCULATE REGRESSION METRICS
    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    rmse = mean_squared_error(y_test, predictions, squared=False)  # Calculate RMSE
    r2 = r2_score(y_test, predictions)

    row = {'Model': key,
           'Run Time (minutes)': round((time.time() - start_time) / 60, 2),
           'MAE': mae,
           'MSE': mse,
           'RMSE': rmse,
           'R2': r2
           }

    df_models = pd.concat([df_models, pd.DataFrame([row])], ignore_index=True)

# Sort the DataFrame by R-squared (R2) in descending order
df_models = df_models.sort_values(by='R2', ascending=False)

# PRINT THE MODELS WITH REGRESSION METRICS [SORTED]
print(df_models)

C:\Users\Siddharth Shah\AppData\Local\Temp\ipykernel_16984\201078824.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_models = pd.concat([df_models, pd.DataFrame([row])], ignore_index=True)


              Model  Run Time (minutes)           MAE           MSE  \
0  LinearRegression                0.00  13818.326898  3.996501e+08   
2      RandomForest                0.01  16279.296916  6.187934e+08   
3           XGBoost                0.00  17373.696555  7.403353e+08   
1      DecisionTree                0.00  22946.349398  1.399557e+09   

           RMSE        R2  
0  19991.249438  0.154048  
2  24875.559110 -0.309819  
3  27209.102913 -0.567091  
1  37410.651439 -1.962486  


c:\Users\Siddharth Shah\anaconda3\envs\eightfoldai\Lib\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
